# Tutorial for Downloading Omni6DPose Dataset
Omni6DPose is a dataset for universal 6D object pose estimation and tracking, featured by its diversity in object categories, large scale, and variety in object materials. The project is available at [Omni6DPose](https://jiyao06.github.io/Omni6DPose/). In this tutorial, we will show you how to download the dataset.

In [ ]:
%pip install requests
%pip install tqdm

In [ ]:

############################ Set the configuration for loading data ##########################
DATA_LINKS = 'data_links.json'   # Path to the json file containing the download links
DATA_PATH = 'data/Omni6DPose/'           # Path to the directory where the data will be stored
##############################################################################################

import json, os, sys, requests
from tqdm import tqdm

# Get the download links and create the data directory
with open(DATA_LINKS) as f:
    data = json.load(f)
for key in data:
    os.makedirs(DATA_PATH + key, exist_ok=True)

# Define the download function
def download_file(url, filename):
    progress_bar = None
    try:
        if os.path.exists(filename) :
            print(f"{filename} already exists!")
            return True
        else:
            print('\033[93m' + f"Downloading {filename}..." + '\033[0m')
        url = url.replace('dl=0', 'dl=1')
        response = requests.get(url, stream=True)     
        total_size_in_bytes = int(response.headers.get('content-length', 0))
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)   
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    progress_bar.update(len(chunk))
                    f.write(chunk)
            progress_bar.close()
            return True
        else:
            print('\033[91m' + f"Failed to download {filename}! The server returned status code {response.status_code}" + '\033[0m')
            return False
    
    except (KeyboardInterrupt, requests.RequestException, Exception) as e:
        if os.path.exists(filename):
            os.remove(filename)
        print('\033[91m' + f"An error occurred while downloading {filename}! {e} The file has been removed." + '\033[0m')
        return False
    
    finally:
        if progress_bar is not None:
            progress_bar.close()

- ### 1. Download Meta files
    The Meta files are available at [Meta](https://www.dropbox.com/scl/fo/xumdcn3yp4hi0ahrg0v23/AMEANjZJoDpg5eeccQ02KnQ?rlkey=csw09o0uoh2q6ia93wcpp3mea&st=sru933q5&dl=0). You can download the files using the following command. The files is about 4MB.

In [ ]:
Meta_links = data['Meta']
for item, link in Meta_links.items():
    print('\033[93m' + f"Downloading {item} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'Meta', exist_ok=True)
    download_file(link, f"{DATA_PATH}Meta/{item}")

- ### 2. Download PAM dataset [Optional]
    The PAM dataset is available at [PAM](https://www.dropbox.com/scl/fo/oxpad3hjijwehrbdp0krf/AHaWA2xhz0lN-x6elq3Sf2I?rlkey=sctuyqczvmqukth4z4lqk2uj7&st=xx2xpwky&dl=0). You can download the dataset using the following command. The dataset is about 14GB. PAM contains the pose aligned 3D models, which is not necessary for the benchmarking if your method does not require the reference 3D models.

In [ ]:
PAM_links = data['PAM']
for item, link in PAM_links.items():
    print('\033[93m' + f"Downloading {item} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'PAM', exist_ok=True)
    download_file(link, f"{DATA_PATH}PAM/{item}")

- ### 3. Download ROPE dataset
    The ROPE dataset is available at [ROPE](https://www.dropbox.com/scl/fo/). You can download the dataset using the following command. The dataset contains 363 scenes, with each scene being approximately 500MB. The total size of the dataset is about 180GB. If you want to download a subset of the dataset, you can set the parameter `SCENE_NUM` in the following scripts. If the download is interrupted, you can rerun the script, and it will skip the downloaded files.

In [ ]:
################################### Set the configuration for loading the ROPE data ###################################
FILES = ['meta.zip', 'color.zip', 'depth.zip', 'mask.zip', 'mask_sam.zip']   # The files to be downloaded
SCENE_NUM = 2                                                                # The number of scenes to be downloaded
#######################################################################################################################

assert SCENE_NUM > 0 and SCENE_NUM <= 363, 'Invalid scene number'
ROPE_links = data['ROPE']
scenes = list(ROPE_links.keys())
scene_ids = [scene.split('-')[0] for scene in scenes]
scene_ids = sorted(list(set(scene_ids)))[:SCENE_NUM]
print(f"The selected scenes are: {scene_ids}")

for scene in scene_ids:
    print('\033[93m' + f"Downloading scene {scene} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'ROPE/' + scene, exist_ok=True)
    for file in FILES:
        download_file(ROPE_links[f"{scene}-{file}"], f"{DATA_PATH}ROPE/{scene}/{file}")

- ### 4. Download SOPE dataset
    The SOPE dataset is available at [SOPE](). You can download the dataset using the following command. The dataset comprises 51 patches. If you want to download a subset of the dataset, you can set the `PATCH_NUM` parameter in the scripts below. If the download is interrupted, you can rerun the script, and it will skip the downloaded files.
    
    The files `ir.zip`, `depth_1.zip`, and `coord.zip` are not necessary for benchmarking. Specifically, `ir.zip` contains the simulated infrared images, `depth_1.zip` includes the perfect depth images, and `coord.zip` contains the NOCS map proposed in [NOCS](). If you wish to download these files, you can set the `DOWNLOAD_IR`, `DOWNLOAD_GT_DEPTH`, and `DOWNLOAD_NOCS` parameters to `True` in the scripts below. Each batch of the dataset without the optional files is about 14GB. The total size of the dataset is about 720GB.

In [ ]:
############################################ Set the configuration for loading the SOPE data ############################################
FILES = ['meta.zip', 'color.zip', 'depth.zip', 'depth_1.zip', 'mask.zip', 'coord.zip', 'ir.zip'] # The selected files to be downloaded
DOWNLOAD_IR = False                                                                              # Download the IR images
DOWNLOAD_GT_DEPTH = False                                                                        # Download the perfect depth images
DOWNLOAD_NOCS = False                                                                            # Download the NOCS map
PATCH_NUM = 2                                                                                    # The number of patches to be downloaded
#########################################################################################################################################

FILES.remove('ir.zip') if not DOWNLOAD_IR else None
FILES.remove('depth_1.zip') if not DOWNLOAD_GT_DEPTH else None
FILES.remove('coord.zip') if not DOWNLOAD_NOCS else None

assert PATCH_NUM > 0 and PATCH_NUM <= 50, 'Invalid patch number'
SOPE_links = data['SOPE']
patches = list(SOPE_links.keys())
patch_ids = [patch.split('-')[0] for patch in patches]
patch_ids = sorted(list(set(patch_ids)))[:PATCH_NUM]
print('\033[93m' + f"The total number of patches is: {len(patch_ids)}" + '\033[0m')
print('\033[93m' +  f'The selected patches are: {patch_ids}' + '\033[0m')

for patch in patch_ids:
    print('----------------------------------------------------------------------------------------------------')
    print('\033[93m' + f"Downloading patch {patch} ..." + '\033[0m')
    os.makedirs(DATA_PATH + 'SOPE/' + patch + '/train', exist_ok=True)
    os.makedirs(DATA_PATH + 'SOPE/' + patch + '/test', exist_ok=True)
    for file in FILES:
        status = download_file(SOPE_links[f"{patch}-{'train'}-{file}"], f"{DATA_PATH}SOPE/{patch}/train/{file}")
        sys.exit() if not status else None
        status = download_file(SOPE_links[f"{patch}-{'test'}-{file}"], f"{DATA_PATH}SOPE/{patch}/test/{file}")
        sys.exit() if not status else None
    print('----------------------------------------------------------------------------------------------------')
print('\033[92m' + "All data downloaded successfully!" + '\033[0m')

- ### 5. Unzip the files
    After downloading the files, you can unzip the files using the following command. If you want to remove the zip files after unzipping, you can set the `REMOVE_ZIP` parameter to `True` in the scripts below. The files will be unzipped to the `./data` directory, and the directory structure will be as follows:
    ```
    data
    └── Omni6DPose
        ├── ROPE
        │   ├── SCENE_ID
        │   │   ├── FRAME_ID_meta.json
        │   │   ├── FRAME_ID_color.png
        │   │   ├── FRAME_ID_mask.exr
        │   │   ├── FRAME_ID_depth.exr
        │   │   ├── FRAME_ID_mask_sam.npz [Optional]
        │   │   └── ...
        │   └── ...
        ├── SOPE
        │   ├── PATCH_ID
        │   │   ├── train
        │   │   │   ├── SCENE_NAME
        │   │   │   |  ├── SCENE_ID
        │   |   |   |  |   ├── FRAME_ID_meta.json
        │   │   │   |  |   ├── FRAME_ID_color.png
        │   │   │   |  |   ├── FRAME_ID_mask.exr
        │   |   |   |  |   ├── FRAME_ID_depth.exr
        │   │   │   |  |   ├── FRAME_ID_depth_1.exr [Optional]
        │   │   │   |  |   ├── FRAME_ID_coord.png   [Optional]
        │   │   │   |  |   ├── FRAME_ID_ir_l.png    [Optional]
        │   │   │   |  |   ├── FRAME_ID_ir_r.png    [Optional]
        │   │   │   |  |   └── ...
        │   │   │   |  └── ...
        │   │   │   └── ...
        │   │   └── test
        │   └── ...
        ├── PAM
        │   └── obj_meshes
        │       ├── DATASET-CLASS_ID
        │       └── ...
        └── Meta    
            ├── obj_meta.json 
            └── real_obj_meta.json
    ```


In [ ]:
import zipfile
import os
from tqdm import tqdm

################################### Set the configuration for unzipping the downloaded files ##################################
REMOVE_ZIPS = True   # Remove the zip files after unzipping
UNZIP_PAM = True     # Unzip the PAM files
UNZIP_ROPE = True    # Unzip the ROPE files
UNZIP_SOPE = True    # Unzip the SOPE files
###############################################################################################################################

def unzip_files(directory, remove_zips=False):
    files = os.listdir(directory)
    for file in files:
        if file.endswith('.zip'):
            with zipfile.ZipFile(directory + file, 'r') as zip_ref:
                zip_ref.extractall(directory)
            if remove_zips:
                os.remove(directory + file)

# Unzip the PAM files
if UNZIP_PAM:
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'PAM/'} ..." + '\033[0m')
    unzip_files(DATA_PATH + 'PAM/', remove_zips=REMOVE_ZIPS)

# Unzip the ROPE files
if UNZIP_ROPE:
    scenes = os.listdir(DATA_PATH + 'ROPE/')
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'ROPE/'} ..." + '\033[0m')
    for scene in tqdm(scenes):
        unzip_files(DATA_PATH + 'ROPE/' + scene + '/', remove_zips=REMOVE_ZIPS)
    
# Unzip the SOPE files
if UNZIP_SOPE:
    patches = os.listdir(DATA_PATH + 'SOPE/')
    print('\033[93m' + f"Unzipping files in {DATA_PATH + 'SOPE/'} ..." + '\033[0m')
    for patch in tqdm(patches):
        print('\033[93m' + f"Unzipping files in {DATA_PATH + 'SOPE/' + patch} ..." + '\033[0m')
        unzip_files(DATA_PATH + 'SOPE/' + patch + '/train/', remove_zips=REMOVE_ZIPS)
        unzip_files(DATA_PATH + 'SOPE/' + patch + '/test/', remove_zips=REMOVE_ZIPS)
    print('\033[92m' + "All files unzipped successfully!" + '\033[0m')